In [ ]:
"""Unit for melody generator, taking the trained model"""

'Unit for melody generator, taking the trained model'

In [ ]:
import json
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

from dataPreprocessor import SEQUENCE_LENGTH, MAPPING_JSON_NAME
import music21 as m21
import os

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR) #for avoiding annoying tf warnings


MODEL_PATH = "model.h5"

class Generator:
    """Class for generating music from sequences with a trained model"""
    def __init__(self, model_path = MODEL_PATH):
        """Model initializer"""
        self.model_path = model_path
        self.model = keras.models.load_model(model_path)
        self._start_symbols = ["/"] * SEQUENCE_LENGTH #indicates the beginning of a new song


        #loads the mapping json file..
        with open(MAPPING_JSON_NAME, "r") as fp:
            self._mappings = json.load(fp)
            
    def generate_music(self, seed, num_steps, max_sequence_length, temperature):
        """ Generator of music from a seed sequence
        args:
        seed: sequence of music that will be feeded to the model. It will have the form:
        start_symbol + seed (adding the start symbol to indicate the begining of the seed)
        num_steps: how many steps will have the output (continuation of feeded idea) in semicorcheas
        max_sequence_length: maximum length of the incoming sequence feed. In this case
        this will be 64 steps (16 semicorcheas) or four bars or 4/4
        temperature: number from 0-1. The higher the more random the note choice
        
        return: melody!"""

        #print(self._mappings)
        #Read the seed and structure it with the start symbol
        seed = seed.split() #split() takes a string and generates a list of all elements separated
        melody_generated = seed #la melodia a generar sera la original mas the predicted notes 

        seed = self._start_symbols + seed #takes the structure
        #Convert to integers using mapping
        seed = [self._mappings[symbol] for symbol in seed]
        ##Prediction loop: ingresaremos repetidas veces el seed, ira prediciendo una siguiente nota
        #Agregamos esa nota al seed y volvemos a ingresarlo para que prediga otra y assi durante num_steps

        for _ in range(num_steps):
            #Limit the length of the seed...
            seed = seed[-max_sequence_length:]

            #One-hot encode the seed
            one_hot_seed = keras.utils.to_categorical(seed, num_classes= len(self._mappings))

            #Modificar el shape del seed para que se adapte a la dimension que necesita Keras
            #en orden a predecir: (1, max_sequence_length, num symbols on vocabulary). Para ello
            #usamos newaxis (newaxis, one_hot_seed), que agrega una dimension al principio
            one_hot_seed = one_hot_seed[np.newaxis,...]

            #Now keras predicts generating a probability distribution vector...
            probabilities = self.model.predict(one_hot_seed)[0]

            #Using temperature gives the chance of picking a random note different from the obvious
            #which is the one that the model assigns the highest prob. If temperature is zero, we pick
            #that one, but with higher values appears a random choice. For this we use the sample_with_temperature
            #method

            output_int = self._sample_with_temperature(probabilities, temperature)
            #output_int = np.argmax(probabilities)

            #Now I update the seed adding the predicted note
            seed.append(output_int)

            #Convierto este valor que acabo de predecir al simbolo original  en el dicccionario
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            #Si el simbolo que se obtiene es un"/", quiere decir que llegamos al final y paramos
            if output_symbol == "/":
                break

            else:
            #Agregamos el symbolo a la melodia que se esta generando...
                melody_generated.append(output_symbol)

        return melody_generated


    def _sample_with_temperature(self, probabilities, temperature):
        """Using temperature gives the chance of picking a random note different from the obvious
        which is the one that the model assigns the highest prob. If temperature is zero, we pick
        that one, but with higher values appears a random choice.
        
        By dividing log(probabilities_vector) /temperature, if temperature is closer to 1 this
        value will be smaller, hence the softmax of this will be softened (with more similar values)
        On the contrary, with temp closer to zero this difference will accentuate 
        args:
        probabilities (ndarray): array containing the probability of each possible outcome
        temperature: float in interval [0,1]. Number closer to 0 makes the model more deterministic,
        A number closer to 1  makes the generation more impredictable
            
            return: selected output symbol """
        predictions = np.log(probabilities) / temperature
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions)) #Softmax vector

        choices = range(len(probabilities)) #[0,1,2,3,4,5,....] posible outcomes (espacio muestral)
        index = np.random.choice(choices, p = probabilities)
        return index


    #Convert melody into midi file
    def convert_into_midi(self, melody, step_duration = 0.25, format = "midi", file_name = "mel.mid"):
        """Utility that converts melody file into a m21 object and
        then into a midi format
        args:
        melody (list): song in a form of list of notes/rests
        step_duration (float): 0.25 - semicorcheas as the sequence step
        format (str): midi format in this case
        file_name (str): name of the final midi file
        returns: converted midi file"""

        #Create a stream object as the container for the converted song
        stream = m21.stream.Stream()

        #Parse  all the symbols  in the melody and create note/rest m21 objects
        #Go through all the elements of the list and convert them into notes/rests.

        #Ex: 60 _ _ _ r _ 62 _
        #Se recorre toda la cadena comenzando por el 60 inicial. Como luego hay un _
        #se continua hasta que nos topamos con el "r". Ahi entonces miramos hacia atras
        #y resolvemos la nota que se toco (60) y la duracion total, que hemos ido 
        #acumulando hasta toparnos con el "r"

        #Initialize variables
        start_symbol = None #the first event in the chain
        step_counter = 1 #acumulates the duration of the current event until it changes
        #Ex if the step_counter is 4 it means that the duration is 1 quarter note (negra)
        
        #Begins moving through all the elements of melody list
        for  i, symbol in enumerate(melody):
            #CASE 1: If the symbol is a note/rest...
            if symbol != "_" or (i + 1) == len(melody): #es decir, si hemos llegado a un cambio y aparece un a nota o rest,     resolvemos #O bien si hemos llegado al final (i +1) == len(melody), ahi también resolvemos
                
                #Make sure that we are not in the first symbol of the list
                if start_symbol is not None:#si no estoy partiendo sino que ya voy en el segundo evento por ej.
                    #Calculate the duration of the last event
                    
                    quarter_length_duration = step_counter * step_duration
                    #If start_symbol vigente is a rest
                    if start_symbol == "r": #si el evento en el que íbamos era un "r"
                        m21_event = m21.note.Rest(quarterLength = quarter_length_duration) #registramos r y su duracion 

                    #If start_symbol  vigente is a note, guarda esa nota y su duracion acumulada
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength = quarter_length_duration) 

                    #En ambos casos append al stream
                    stream.append(m21_event)

                    #Reseteamos el step_counter para empezar a contar de nuevo el duration
                    step_counter = 1

                #Update the start_symbol con el nuevo evento
                start_symbol = symbol #queda en el symbol que ahora topamos

            
            
            #CASE 2: else : If the symbol is a prolongation "_"...
            else: #we  continue acumulating the duration of the event
                step_counter += 1

        #Write the m21 stream to a midi file
        stream.write(format, file_name)
        print("Midi file generated")




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if __name__ == "__main__":
    mg = Generator()
    seed_x = "67 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"
    melody = mg.generate_music(seed = seed_x, num_steps = 500, max_sequence_length= SEQUENCE_LENGTH, temperature = 0.9)
    mg.convert_into_midi(melody, step_duration = 0.25, format = "midi", file_name = "mel.mid")

Midi file generated
